In [1]:
# If any of the Packages is missing, download them here
# using Pkg
# Pkg.add("CSV")
# Pkg.add("DataFrames")
# Pkg.add("Clustering")
# Pkg.add("GaussianMixtures")
# Pkg.add("Distances")
# Pkg.add("Statistics")
# Pkg.add("ScikitLearn")

In [2]:
using CSV, DataFrames, Clustering, GaussianMixtures, Distances, Statistics
using ScikitLearn: @sk_import

In [3]:
# Load the DataSet
file = CSV.File("01_hdp.csv") |> DataFrame

,tumorsize,co2,pain,wound,mobility,ntumors,nmorphine,remission,lungcapacity
,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Float64
1,67.9812,1.53433,4,4,2,0,0,0,0.801088
2,64.7025,1.67613,2,3,2,0,0,0,0.326444
3,51.567,1.53345,6,3,2,0,0,0,0.565031
4,86.438,1.4533,3,3,2,0,0,0,0.848411
5,53.4002,1.56635,3,4,2,0,0,0,0.886491
6,51.6573,1.41787,4,5,2,0,0,0,0.701031
7,78.9171,1.70663,3,4,2,0,0,0,0.890854
8,69.8332,1.52873,3,3,3,0,0,0,0.66088
9,62.8526,1.54296,4,4,3,2,0,0,0.908871


In [4]:
# Preparing the Encoder
@sk_import preprocessing : LabelEncoder
enc = LabelEncoder()

PyObject LabelEncoder()

In [5]:
# Encoding all categorical columns
file[!,"FamilyHx"] .= enc.fit_transform(file[!,"FamilyHx"])
file[!,"SmokingHx"] .= enc.fit_transform(file[!,"SmokingHx"])
file[!,"Sex"] .= enc.fit_transform(file[!,"Sex"])
file[!,"CancerStage"] .= enc.fit_transform(file[!,"CancerStage"])
file[!,"School"] .= enc.fit_transform(file[!,"School"])

8525-element Array{Int64,1}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [6]:
# Normalizing the columns, dividing all the rows by its maximun value
for i=1:size(file)[2]
    file[!,i] = file[!,i] ./ findmax(file[!,i])[1]
end

In [7]:
# transforming dataframe to array
file = convert(Array, file)

8525×27 Array{Float64,2}:
 0.58374   0.720983  0.444444  …  0.0  0.333333  0.0285714  0.740008
 0.555587  0.787615  0.222222     0.0  0.333333  0.0285714  0.740008
 0.442795  0.720566  0.666667     0.0  0.333333  0.0285714  0.740008
 0.742225  0.682906  0.333333     0.0  0.333333  0.0285714  0.740008
 0.458536  0.736027  0.333333     0.0  0.333333  0.0285714  0.740008
 0.44357   0.666256  0.444444  …  0.0  0.333333  0.0285714  0.740008
 0.677645  0.801947  0.333333     0.0  0.333333  0.0285714  0.740008
 0.599644  0.718348  0.333333     0.0  0.333333  0.0285714  0.740008
 0.539702  0.725038  0.444444     0.0  0.333333  0.0285714  0.740008
 0.616342  0.748581  0.555556     0.0  0.333333  0.0285714  0.740008
 0.566948  0.75624   0.444444  …  0.0  0.333333  0.0285714  0.740008
 0.493919  0.768006  0.444444     0.0  0.333333  0.0285714  0.740008
 0.419374  0.721614  0.555556     0.0  0.333333  0.0285714  0.740008
 ⋮                             ⋱                 ⋮          
 0.659156  0.711

In [8]:
# Finding the Eucledian distance for the transposed array
dist = pairwise(Euclidean(), file', dims=2)

8525×8525 Array{Float64,2}:
 0.0       1.18056   1.62335   0.580707  …  2.07547  2.27387  1.854
 1.18056   0.0       1.33318   1.22982      1.92066  2.19072  2.19346
 1.62335   1.33318   0.0       1.70824      2.22262  2.27824  2.35196
 0.580707  1.22982   1.70824   0.0          2.13501  2.4488   1.90259
 0.641326  1.27128   1.55991   0.709556     2.14618  2.23717  1.77964
 1.22838   1.62857   1.23331   1.20754   …  2.3694   2.56994  2.07698
 1.53458   1.32139   1.26001   1.58008      2.1426   2.47601  2.49828
 0.58301   1.13888   1.63696   0.660828     2.11593  2.33068  1.91716
 1.46354   1.89831   1.61994   1.53988      2.53327  2.66027  2.32604
 0.668771  1.3693    1.55261   0.813461     2.13833  2.19403  1.799
 1.24171   0.820086  1.54915   1.16264   …  1.91983  2.36086  2.30813
 1.12442   0.404464  1.26319   1.20855      1.84321  2.09569  2.16271
 1.54179   1.32493   0.627768  1.62607      2.1276   2.21834  2.29272
 ⋮                                       ⋱                    
 1.

In [9]:
# Clustering the data into 10 clusters using K-means
print("Time to load Kmeans Cluster:")
@time Kmeans = kmeans(file', 10)

Time to load Kmeans Cluster:  3.375848 seconds (8.21 M allocations: 395.458 MiB, 0.65% gc time)


KmeansResult{Array{Float64,2},Float64,Int64}([0.594897914535297 0.5921042850682409 … 0.6321318656415945 0.5798250116905681; 0.7477308493448604 0.7461919173909755 … 0.7718909517803058 0.7398658897936501; … ; 0.5021344040574807 0.4929333177077221 … 0.5136938391175679 0.48192056905749825; 0.6149548282488102 0.6139408021713405 … 0.6275475351993813 0.6229949030226087], [1, 1, 2, 1, 1, 5, 2, 1, 9, 1  …  5, 8, 2, 1, 9, 5, 8, 3, 3, 3], [1.4011775419588908, 1.5648519162143266, 1.2037902118788537, 1.6985773445470969, 1.3952768132218711, 1.2147805772060885, 1.526689080597647, 1.4797533198678536, 1.3732965847736835, 1.2778132411761103  …  1.3213321809616403, 1.1864210129822297, 1.651109382201998, 1.5224803283355737, 3.072147418131916, 1.7758553722016686, 1.3463388820983013, 1.032053994757348, 1.6536768716679475, 1.3291890442254015], [1352, 1219, 867, 476, 1208, 715, 448, 745, 531, 964], [1352, 1219, 867, 476, 1208, 715, 448, 745, 531, 964], 8980.980811368008, 13, true)

In [10]:
# Scoring the clustering model
# A smaller Total Cost is always better
# The Silhouette Coefficient has a range of -1 to 1, -1 being the worst possible accuracy and 1 being the best
print("Score for Kmeans: 
----------------------------
Total Cost: $(Kmeans.totalcost) 
Silhouettes Coefficient: $(mean(silhouettes(Kmeans.assignments, Kmeans.counts, dist)))")

Score for Kmeans: 
----------------------------
Total Cost: 8980.980811368008 
Silhouettes Coefficient: 0.14767512823988224

In [11]:
# Finding the Outliers AKA the max distances from the centers
sil = silhouettes(Kmeans.assignments, Kmeans.counts, dist)
ind = 1
for i in partialsortperm(sil, 1:3)
    println("$(ind). Outlier was detected in cluster number $(Kmeans.assignments[i]), at point number $i with a silhouette coefficient of $(sil[i])")
    ind = ind + 1
end

1. Outlier was detected in cluster number 9, at point number 1318 with a silhouette coefficient of -0.08993843282911018
2. Outlier was detected in cluster number 9, at point number 4543 with a silhouette coefficient of -0.07422749124889305
3. Outlier was detected in cluster number 9, at point number 6509 with a silhouette coefficient of -0.0696821051093548


In [12]:
# Clustering the data into 22 clusters using K-medoids
print("Time to load Kmedoids Cluster:")
@time Kmedoids = kmedoids(dist, 10)

Time to load Kmedoids Cluster:  0.755929 seconds (549.89 k allocations: 305.067 MiB, 20.02% gc time)


KmedoidsResult{Float64}([4857, 5725, 3131, 6033, 7648, 4204, 2225, 4085, 4095, 2547], [7, 8, 9, 7, 7, 3, 9, 7, 1, 7  …  6, 4, 9, 8, 1, 6, 4, 4, 8, 2], [1.1383991709104617, 1.3689054160410694, 1.1557150618491494, 1.318498600660684, 1.0013523782229456, 0.8640552307260246, 1.490916825473896, 1.1894104584457563, 1.1769819675107644, 0.9531899823028017  …  1.0088396988526795, 1.0390009139103757, 1.3481983709230614, 1.3412710006846238, 1.6560059026655227, 1.1895043933326832, 1.1569903834951005, 1.2803711393528838, 1.619090491808629, 1.3356998180681923], [793, 489, 686, 1194, 390, 786, 494, 1377, 1693, 623], 9674.043543643125, 4, true)

In [13]:
# Scoring the clustering model
# A smaller Total Cost is always better
# The Silhouette Coefficient has a range of -1 to 1, -1 being the worst possible accuracy and 1 being the best
print("Score for Kmedoids: 
----------------------------
Total Cost: $(Kmedoids.totalcost) 
Silhouettes Coefficient: $(mean(silhouettes(Kmedoids.assignments, Kmedoids.counts, dist)))")

Score for Kmedoids: 
----------------------------
Total Cost: 9674.043543643125 
Silhouettes Coefficient: 0.11029004019660318

In [14]:
# Finding the Outliers AKA the max distances from the centers
sil = silhouettes(Kmedoids.assignments, Kmedoids.counts, dist)
ind = 1
for i in partialsortperm(sil, 1:3)
    println("$(ind). Outlier was detected in medoid number $(Kmedoids.assignments[i]), at point number $i with a silhouette coefficient of $(sil[i])")
    ind = ind + 1
end

1. Outlier was detected in medoid number 1, at point number 5328 with a silhouette coefficient of -0.25099556757009145
2. Outlier was detected in medoid number 1, at point number 4724 with a silhouette coefficient of -0.24166391205965643
3. Outlier was detected in medoid number 1, at point number 5343 with a silhouette coefficient of -0.23581903246817082


In [15]:
# Clustering the data using Dbscan with a radius of 0.99 and a minimum number of neighboring points of 4
print("Time to load Dbscan Cluster:")
@time Dbscan = dbscan(dist, 0.99, 4)

Time to load Dbscan Cluster:  0.178824 seconds (121.31 k allocations: 42.639 MiB)


DbscanResult([1, 2, 3, 6, 9, 15, 19, 22, 33, 36  …  144, 165, 258, 267, 284, 288, 336, 366, 911, 1180], [1, 2, 3, 1, 1, 4, 3, 1, 5, 1  …  10, 11, 3, 2, 30, 10, 11, 9, 9, 13], [555, 797, 1218, 811, 241, 371, 274, 170, 418, 457  …  137, 50, 130, 79, 104, 12, 17, 9, 53, 31])

In [16]:
# Silhouettes does not accept that some points are not in the clusters so I assign them to the first cluster
for i=1:size(Dbscan.assignments)[1]
    if Dbscan.assignments[i] == 0
        Dbscan.assignments[i] = 1
        Dbscan.counts[1] = Dbscan.counts[1] + 1
    end
end

In [17]:
# Scoring the clustering model
# The Silhouette Coefficient has a range of -1 to 1, -1 being the worst possible accuracy and 1 being the best
print("Score for Dbscan: 
----------------------------
Silhouettes Coefficient: $(mean(silhouettes(Dbscan.assignments, Dbscan.counts, dist)))")

Score for Dbscan: 
----------------------------
Silhouettes Coefficient: 0.21502827685357004

In [18]:
# Finding the Outliers AKA the max distances from the starts 
sil = silhouettes(Dbscan.assignments, Dbscan.counts, dist)
ind = 1
for i in partialsortperm(sil, 1:3)
    println("$(ind). Outlier was detected in cluster number $(Dbscan.assignments[i]), at point number $i with a silhouette coefficient of $(sil[i])")
    ind = ind + 1
end

1. Outlier was detected in cluster number 1, at point number 3717 with a silhouette coefficient of -0.5510410953549703
2. Outlier was detected in cluster number 1, at point number 2118 with a silhouette coefficient of -0.4646722344363998
3. Outlier was detected in cluster number 1, at point number 3211 with a silhouette coefficient of -0.4424075705688798


In [19]:
# Clustering the data using Fuzzy C-means algorithm with 6 fuzzy clusters and a cluster fuzziness of 2
print("Time to load Fuzzy C-means Cluster:")
@time Fuzzy = fuzzy_cmeans(file', 6, 2)

Time to load Fuzzy C-means Cluster:  1.288878 seconds (2.51 M allocations: 311.136 MiB, 12.75% gc time)


FuzzyCMeansResult: 6 clusters for 8525 points in 27 dimensions (converged in 5 iterations)

In [20]:
# Distance Matrix from the centers to the data
dmat = pairwise(Euclidean(), Fuzzy.centers, file', dims=2)
# Finding sizes of Distance Matrix
k, n = size(dmat)
# Initializing costs and assingments vectors
Fuzzy_costs = zeros(n)
Fuzzy_assignments = Vector{Int}(undef, n)
Fuzzy_counts = fill(0, (k))

6-element Array{Int64,1}:
 0
 0
 0
 0
 0
 0

In [21]:
# Implementing a modified version of the method used on the documentation to find the costs, counts and assignments.
@inbounds for j = 1:n
    # find the closest cluster to the i-th point. Note that a
    # is necessarily between 1 and size(dmat, 1) === k as a result
    # and can thus be used as an index in an `inbounds` environment
    c, a = findmin(view(dmat, :, j))
    # set costs, counts and assingments accordingly
    Fuzzy_assignments[j] = a
    Fuzzy_costs[j] = c
    Fuzzy_counts[a] += 1
end

In [22]:
# Scoring the clustering model
# A smaller Total Cost is always better
# The Silhouette Coefficient has a range of -1 to 1, -1 being the worst possible accuracy and 1 being the best
print("Score for Fuzzy C-means: 
----------------------------
Total Cost: $(sum(Fuzzy_costs)) 
Silhouettes Coefficient: $(mean(silhouettes(Fuzzy_assignments, Fuzzy_counts, dist)))")

Score for Fuzzy C-means: 
----------------------------
Total Cost: 11553.558663701373 
Silhouettes Coefficient: 0.09457012354440009

In [23]:
# Finding the Outliers AKA the max distances from the centers
sil = silhouettes(Fuzzy_assignments, Fuzzy_counts, dist)
ind = 1
for i in partialsortperm(sil, 1:3)
    println("$(ind). Outlier was detected in fuzzy cluster number $(Fuzzy_assignments[i]), at point number $i with a silhouette coefficient of $(sil[i])")
    ind = ind + 1
end

1. Outlier was detected in fuzzy cluster number 2, at point number 5651 with a silhouette coefficient of -0.16029527744161487
2. Outlier was detected in fuzzy cluster number 2, at point number 4228 with a silhouette coefficient of -0.15942543401435727
3. Outlier was detected in fuzzy cluster number 2, at point number 5720 with a silhouette coefficient of -0.15700598284214995


In [24]:
# Clustering the data using Gaussian Mixtures algorithm with 300 gaussians
print("Time to load Gaussian Mixtures")
@time begin
    # Creating the Model
    Gaussian = GMM(50, size(file)[2])
    # Adding the data
    em!(Gaussian, file; nIter= 20, varfloor=1e-10)
end

Time to load Gaussian Mixtures  4.596819 seconds (9.70 M allocations: 606.823 MiB, 1.43% gc time)


20-element Array{Float64,1}:
 -1.0963946383117202
  0.27173994020545034
  0.2717399402054532
  0.2717399402054548
  0.27173994020545583
  0.27173994020545356
  0.27173994020545117
  0.27173994020545383
  0.2717399402054574
  0.2717399402054506
  0.271739940205453
  0.2717399402054602
  0.2717399402054506
  0.2717399402054537
  0.27173994020545794
  0.2717399402054542
  0.27173994020545644
  0.27173994020546105
  0.2717399402055701
  0.2717399402069869

In [25]:
# Scoring the clustering model
# A bigger Average Log Likelihood is always better
print("Score for Gaussian Mixtures: 
----------------------------
Average Log Likelihood: $(avll(Gaussian, file))")

Score for Gaussian Mixtures: 
----------------------------
Average Log Likelihood: 0.27173994022561465

In [26]:
# All logs likelihoods
log = llpg(Gaussian, file)
mins = fill((0.0,0,0),8525)
# Finding the minimun log likelihood for each row
for i=1:size(log)[1]
    min = findmin(log[i,:])
    tuple = (min[1],min[2],i)
    mins[i] = tuple
end

In [27]:
# Finding the Outliers AKA the points with a worse loh likelihood 
ind = 1
for i in partialsortperm(mins, 1:3)
    println("$(ind). Outlier was detected in gaussian number $(mins[i][2]), at row number $(mins[i][3]) with a log-likelihood of $(mins[i][1])")
    ind = ind + 1
end

1. Outlier was detected in gaussian number 1, at row number 8051 with a log likelihood of -27.603600552854303
2. Outlier was detected in gaussian number 1, at row number 3101 with a log likelihood of -25.05602711263232
3. Outlier was detected in gaussian number 1, at row number 913 with a log likelihood of -21.833534603946944


In [28]:
# PLotting to be able to interpret the data, not necessary for the assingment
# So I just comment it

# using Pkg
# Pkg.add("Plots")
#using Plots

In [29]:
#@sk_import manifold : TSNE

In [30]:
# Making the data 2d so it can be plotted

#x = TSNE(n_components=2).fit_transform(file)

In [31]:
# Plotting Kmeans

#a = scatter(x[:,1], x[:,2], marker_z=Kmeans.assignments, color=:seaborn_bright)

In [32]:
# plotting Kmedoids

#b = scatter(x[:,1], x[:,2], marker_z=Kmedoids.assignments, color=:seaborn_bright)

In [33]:
# PLotting Dbscan

#c = scatter(x[:,1], x[:,2], marker_z=Dbscan.assignments, color=:seaborn_bright)

In [34]:
# Plotting Fuzzy C-means

#d = scatter(x[:,1], x[:,2], marker_z=Fuzzy_assignments,color=:seaborn_bright)